In [12]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

root_dir = '../dataset/dna_dataset'
sub_dir = 'relative_dna_dnaseq_intensity'
madf = pd.read_csv(os.path.join(root_dir, 'relative_dna_proteinseq.txt'), sep='\t')

unique_pseq = np.unique(madf['protein_sequence'].values)
print(len(unique_pseq))
print(len(madf))
madf.head(10)

1154
2240


,Motif_ID,Family_Name,protein_sequence,probe_intensity_file
0,M02500_2.00,TCP,TGRKDRHSKVCTAKGPRDRRVRLSAPTAIQFYDVQDRLGFDRPSKA...,M02500_2.00:Zoo_01:6104.txt
1,M02500_2.00,TCP,TGRKDRHSKVCTAKGPRDRRVRLSAPTAIQFYDVQDRLGFDRPSKA...,M02500_2.00:Zoo_01:6113.txt
2,M01812_2.00,bZIP,EKALKKIRRKIKNKISAQESRRKKKEYMDSLEKKVESCSTENLELR...,M01812_2.00:Zoo_01:1186.txt
3,M01812_2.00,bZIP,EKALKKIRRKIKNKISAQESRRKKKEYMDSLEKKVESCSTENLELR...,M01812_2.00:Zoo_01:126.txt
4,M01674_2.00,AT hook,KRKRGRPRKYEHS,M01674_2.00:Zoo_01:6690.txt
5,M01674_2.00,AT hook,KRKRGRPRKYEHS,M01674_2.00:Zoo_01:6771.txt
6,M01704_2.00,bHLH,PGTKACREKLRREKLNDKFMDLSSVLEPGRTPKTDKSAILDDAIRV...,M01704_2.00:Zoo_01:4421.txt
7,M01704_2.00,bHLH,PGTKACREKLRREKLNDKFMDLSSVLEPGRTPKTDKSAILDDAIRV...,M01704_2.00:Zoo_01:4208.txt
8,M01993_2.00,Forkhead,KAELPYAQLIYKAFMSTPNKAMTLQDIYQWFRENTEKGKSDTKGWQ...,M01993_2.00:Zoo_01:7862.txt
9,M01993_2.00,Forkhead,KAELPYAQLIYKAFMSTPNKAMTLQDIYQWFRENTEKGKSDTKGWQ...,M01993_2.00:Zoo_01:7843.txt


In [14]:
def func(subf):
    f = open(subf, 'r')
    lines = f.readlines()
    vals = list()
    for line in lines:
        tmp = line.rstrip().split('\t')
        # seq, val = tmp[0], tmp[1]
        val = float(tmp[1])
        if not isinstance(val, float):
            print("not float {}".format(val))
            val = 0.0
        vals.append(val)
    vals = np.array(vals)
    return vals

ind = 0
list_vals = []
for k, subdf in tqdm(madf.groupby('protein_sequence')):
    #pseq = subdf['protein_sequence'].values[0]
    
    for f in subdf['probe_intensity_file'].values:
        subf = os.path.join(root_dir, sub_dir, f)
        vals = func(subf)
        list_vals.append(vals)
    ind+=1
ind

100%|██████████| 1154/1154 [00:28<00:00, 40.59it/s]


1154

In [15]:
final_vals = np.concatenate(list_vals)
mean, std = final_vals.mean(), final_vals.std()
print(mean, std)

4834.188361720781 10629.057109104138


In [17]:
final_vals2 = (final_vals - mean) /  std

sz = len(final_vals2)
inds = np.arange(sz)
shuf_inds = np.random.permutation(sz)
final_vals2[inds[shuf_inds[:100]]]

array([-0.34515076, -0.32738076, -0.40559467, -0.29007674, -0.20763454,
        0.54876898,  0.10001012, -0.00829132,  0.37198076, -0.02285666,
       -0.4355076 , -0.44561212, -0.12433379, -0.01812538, -0.20566813,
       -0.16001768, -0.45045108, -0.36708279, -0.37426508, -0.08293262,
       -0.00219208,  0.96916863, -0.25186618,  0.43022424,  0.47007952,
       -0.16328158, -0.34254268, -0.30369227, -0.33981316, -0.10488159,
       -0.32878434,  0.17027745, -0.21249365, -0.36373988, -0.43960175,
        0.85548468, -0.23357441, -0.02171487, -0.4396507 ,  0.92555697,
       -0.42612652, -0.36404533, -0.43245461, -0.44989972, -0.34316451,
       -0.4413976 , -0.13675979, -0.30345274,  0.76480282, -0.24377511,
       -0.0976398 , -0.221019  ,  0.13364337, -0.40543348, -0.28337537,
       -0.26320438, -0.30669551, -0.43368089, -0.44980816, -0.32977641,
       -0.33629816, -0.0271162 , -0.18508274, -0.44642052, -0.35896731,
       -0.42890227,  0.14111202, -0.29730237, -0.21123045,  1.68

In [19]:
from sklearn.cluster import KMeans

bin_info = dict()
# TODO: 
dgs = final_vals2

X = dgs.reshape(-1,1)
# TODO: make n_cluster a parameters
model = KMeans(n_clusters=8)

model.fit(X)
print('fit done')
yhat = model.predict(X)
clusters = np.unique(yhat)
for cluster in tqdm(clusters):
    row_ix = np.where(yhat == cluster)
    cur_x = X[row_ix]
    # cur_x.mean(), cur_x.max(), cur_x.min(), len(cur_x)
    bin_info[cur_x.mean()] = ( cur_x.max(), cur_x.min(), len(cur_x) )

bin_info

/home/megvii/anaconda3/envs/paul-py3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


fit done


100%|██████████| 8/8 [00:00<00:00, 12.08it/s]


{0.9806687660292668: (1.7741456222045462, 0.5540604098584603, 6485921),
 5.061624169511751: (7.29276750596195, 3.8141536971006618, 817975),
 19.49504281811333: (29.29970319548812, 14.508400402676708, 35554),
 -0.3340462116534165: (-0.10315338890988351, -3.6053644228655566, 60999887),
 2.568771040536042: (3.8141479081483802, 1.774145810367991, 1864191),
 9.527953805035844: (14.508210326276398, 7.29279610884448, 156324),
 39.11289265753756: (139.04357695022424, 29.302534212498763, 7512),
 0.12786895172032997: (0.5540603157767379, -0.10315337009353903, 20418471)}